In [32]:
import sys, os
import numpy as np
sys.path.append(os.pardir)  # 为了导入父目录中的文件而进行的设定
import numpy as np



class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss
    
    def numerical_gradient_(self):
        h=0.0001;
        wimd=self.W.flatten();
        grad=np.zeros_like(wimd);
        
        for idx in range(wimd.size):
            tmp_val = wimd[idx]
            wimd[idx] = float(tmp_val) + h;
            fxh1 = f(wimd.reshape(self.W.shape));
            wimd[idx] = tmp_val - h;
            fxh2 = f(wimd.reshape(self.W.shape));
        
            grad[idx] = (fxh1-fxh2)/(2*h)
            wimd[idx] = tmp_val;
        
        return wimd.reshape(self.W.shape)

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()

f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)
print(net.predict(x))
print(f(1))
print(net.numerical_gradient_())
print(dW)


[-0.21230986  0.22287786 -1.59666956]
2.4124544281148963
[[-0.54365871  1.31016551 -1.07640974]
 [ 0.12653929 -0.6258016  -1.0564708 ]]
[[ 0.2146122   0.33163018 -0.54624238]
 [ 0.3219183   0.49744526 -0.81936357]]


In [30]:
del numerical_gradient

In [31]:
import numpy as np

def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 还原值
        
    return grad


def numerical_gradient_2d(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad


def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 还原值
        it.iternext()   
        
    return grad

In [26]:
def softmax(a):
    c=np.max(a);
    return np.exp(a-c)/(np.sum(np.exp(a-c)))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 监督数据是one-hot-vector的情况下，转换为正确解标签的索引
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size